In [8]:
#Dependencies
import pandas as pd
import hvplot.pandas
import numpy as np
import panel as pn
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stats

# filtered_df DataFrame is already defined with the necessary data.

# Load the data
path = 'Averaged_Cleaned_filtered_data.csv'
Heart_disease_df = pd.read_csv(path)

filtered_df = Heart_disease_df

In [9]:
# Unique values for dropdowns and checkboxes
ethnicities = filtered_df['Ethnicity'].unique().tolist()
states = filtered_df['US States'].unique().tolist()
years = sorted(filtered_df['Year'].unique().astype(int).tolist())
age_ranges = filtered_df['Age range'].unique().tolist()
heart_disease_types = filtered_df['Heart Disease Type'].unique().tolist()

In [10]:
# Create interactive widgets

    # CheckBox Widgets
ethnicity_selector_checkbox = pn.widgets.CheckBoxGroup(name='Select Ethnicity', options=ethnicities, value=[ethnicities[0]])
states_selector = pn.widgets.CheckBoxGroup(name='Select States', options=['All States'] + states, inline=True)
states_selector_checkbox = pn.widgets.CheckBoxGroup(name='Select States', options=states, inline=True)
all_states_checkbox = pn.widgets.Checkbox(name='All States', value=True)

    # Drop Down Selectors
ethnicity_selector_dropdown = pn.widgets.Select(name='Select Ethnicity', options=ethnicities, value=ethnicities[0])

start_year_selector = pn.widgets.Select(name='Select Start Year', options=years, value=years[0])
end_year_selector = pn.widgets.Select(name='Select End Year', options=years, value=years[-1])

age_range_selector = pn.widgets.Select(name='Select Age Range', options=age_ranges, value=age_ranges[0])
age_range_selector_dropdown = pn.widgets.Select(name='Select Age Range', options=age_ranges, value=age_ranges[0])

heart_disease_type_selector = pn.widgets.Select(name='Select Heart Disease Type', options=heart_disease_types, value=heart_disease_types[0])
heart_disease_type_selector_dropdown = pn.widgets.Select(name='Select Heart Disease Type', options=heart_disease_types, value=heart_disease_types[0])
    # SLider Selectors
year_range_slider = pn.widgets.IntRangeSlider(name='Select Year Range', start=years[0], end=years[-1], value=(years[0], years[-1]))

In [11]:
# LINE Graph:
# The graph generated is a line plot showing the mortality rates of a selected heart disease type
# for different ethnicities over a chosen year range.

    # Y-axis represents data value per 100,000 people averaged for chosen states.
    # X-axis shows the years within the selected range.

# The lines on the graph are differentiated by ethnicity.

# Dashboard Options:
    # Ethnicity Selector (Checkbox Group): Allows users to select one or more ethnicities.
    # Year Range Slider: Allows users to select a range of years to be displayed on the graph.
    # States Selector (Checkbox Group): Allows users to select specific states to include in the data.
    # All States Checkbox: Allows users to include all states in the data if selected.
    # Age Range Selector (Dropdown): Allows users to select one of two specified age ranges.
    # Heart Disease Type Selector (Dropdown): Allows users to select a specific type of heart disease.

In [12]:
#Creates LineGraph
@pn.depends(ethnicity_selector_checkbox, year_range_slider, states_selector_checkbox, all_states_checkbox, age_range_selector_dropdown, heart_disease_type_selector_dropdown)
def update_plot(selected_ethnicities, selected_year_range, selected_states, all_states_selected, selected_age_range, selected_heart_disease_type):
    if not selected_ethnicities:         # check if at least one ethnicity is selected.
        return "Please select at least one ethnicity."
    
    start_year, end_year = selected_year_range
    
    # Filter data based on the selected ethnicity, age range, heart disease type, and year range.
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'].isin(selected_ethnicities)) &
        (filtered_df['Age range'] == selected_age_range) &
        (filtered_df['Heart Disease Type'] == selected_heart_disease_type) &
        (filtered_df['Year'] >= start_year) &
        (filtered_df['Year'] <= end_year)
    ]
    
    # If "All States" is not selected, filter data based on the selected states.
    if not all_states_selected:
        if not selected_states:
            return "Please select at least one state."
        filtered_data = filtered_data[filtered_data['US States'].isin(selected_states)]
    
    # Group by 'Ethnicity' and 'Year', then calculate the mean of 'Data_Value/100_000 People'
    grouped_data = filtered_data.groupby(['Ethnicity', 'Year'])['Data_Value/100_000 People'].mean().reset_index()
    
    # Creates a line plot showing the mortality rates over the selected years.
    plot = grouped_data.hvplot.line(
        x='Year',
        y='Data_Value/100_000 People',
        by='Ethnicity',
        title=f'{selected_heart_disease_type} Mortality Rates by Ethnicity of {selected_age_range} in {selected_states if not all_states_selected else "All States"}',
        ylabel='Average Data Value per 100,000 People',
        xlabel='Year',
        legend='top',
        width=800,
        height=600
    )
    
    return plot

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    "## Cardiovascular Disease Mortality Rates Linegraph Generator",
    pn.Row(ethnicity_selector_checkbox, year_range_slider),
    pn.Row(all_states_checkbox, states_selector_checkbox),
    pn.Row(pn.Column(age_range_selector_dropdown, heart_disease_type_selector_dropdown)),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] CheckBoxGroup(name='Select Ethnicity', options=['American Indian/Alaska N...], value=['American Indian/Alaska N...])
        [1] IntRangeSlider(end=2019, name='Select Year Range', start=1999, value=(1999, 2019), value_end=2019, value_start=1999)
    [2] Row
        [0] Checkbox(name='All States', value=True)
        [1] CheckBoxGroup(inline=True, name='Select States', options=['AK', 'AL', 'AR', ...])
    [3] Row
        [0] Column
            [0] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
            [1] Select(options=['All heart disease', ...], value='All heart disease')
    [4] ParamFunction(function, _pane=HoloViews, defer_load=False)

In [13]:
# Creates a Panel dashboard for generating a boxplot of cardiovascular disease mortality rates based on user-selected criteria.

# Boxplot Graph:
# The graph generated is a boxplot showing the distribution of mortality rates of a selected heart disease type for different ethnicities.
    # Y-axis - data value per 100,000 people.
    # X-axis - the selected heart disease type, age range, and ethnicity.
    
# The boxplot provides a visual summary of the distribution, central tendency, and variability of the mortality rates.

# Dashboard Options:
    # Ethnicity Selector (Checkbox Group): Allows users to select one or more ethnicities.
    # Year Range Slider: Allows users to select a range of years to be displayed on the graph.
    # States Selector (Checkbox Group): Allows users to select specific states to include in the data.
    # All States Checkbox: Allows users to include all states in the data if selected.
    # Age Range Selector (Dropdown): Allows users to select a specific age range.
    # Heart Disease Type Selector (Dropdown): Allows users to select a specific type of heart disease.

In [14]:
# Creates Boxplots
@pn.depends(ethnicity_selector_checkbox, year_range_slider, states_selector_checkbox, all_states_checkbox, age_range_selector_dropdown, heart_disease_type_selector_dropdown)
def update_plot(selected_ethnicities, selected_year_range, selected_states, all_states_selected, selected_age_range, selected_heart_disease_type):
    if not selected_ethnicities:
        return "Please select at least one ethnicity."
    
    start_year, end_year = selected_year_range

    # Filter data based on selected ethnicity, age range, heart disease type, and year range.
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'].isin(selected_ethnicities)) &
        (filtered_df['Age range'] == selected_age_range) &
        (filtered_df['Heart Disease Type'] == selected_heart_disease_type) &
        (filtered_df['Year'] >= start_year) &
        (filtered_df['Year'] <= end_year)
    ]

    # If "All States" is not selected, filters the data based on the selected states
    if not all_states_selected:
        if not selected_states:
            return "Please select at least one state."
        filtered_data = filtered_data[filtered_data['US States'].isin(selected_states)]
    
    # Creates a boxplot showing the distribution of mortality rates.
    plot = filtered_data.hvplot.box(
        y='Data_Value/100_000 People',
        by=['Heart Disease Type', 'Age range', 'Ethnicity'],
        title=f'{selected_age_range}, {selected_heart_disease_type} Mortality Distribution by Ethnicity in {selected_states}',
        xlabel='Heart Disease Type, Age Range, Ethnicity',
        ylabel='Data Value per 100,000 People',
        legend='top_right',
        rot=45,
        width=800,  # Set the width of the plot
        height=600  # Set the height of the plot
    )
    
    return plot

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    "## Cardiovascular Disease Mortality Rates Boxplot Generator",
    pn.Row(ethnicity_selector_checkbox, year_range_slider),
    pn.Row(all_states_checkbox, states_selector_checkbox),
    pn.Row(pn.Column(age_range_selector_dropdown, heart_disease_type_selector_dropdown)),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] CheckBoxGroup(name='Select Ethnicity', options=['American Indian/Alaska N...], value=['American Indian/Alaska N...])
        [1] IntRangeSlider(end=2019, name='Select Year Range', start=1999, value=(1999, 2019), value_end=2019, value_start=1999)
    [2] Row
        [0] Checkbox(name='All States', value=True)
        [1] CheckBoxGroup(inline=True, name='Select States', options=['AK', 'AL', 'AR', ...])
    [3] Row
        [0] Column
            [0] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
            [1] Select(options=['All heart disease', ...], value='All heart disease')
    [4] ParamFunction(function, _pane=HoloViews, defer_load=False)

In [15]:
# State correlation Heatmap Graph:

# The graph generated is a heatmap showing the correlation between mortality rates of a selected heart disease type for a specific ethnicity and age range across selected states.
    # The color intensity represents the strength of the correlation, with a colorbar for reference.
    # The X and Y axes represent the selected states.

# Dashboard Options:
    # Ethnicity Selector (Dropdown): Allows users to select a specific ethnicity.
    # Age Range Selector (Dropdown): Allows users to select a specific age range.
    # States Selector (Checkbox Group): Allows users to select specific states to include in the correlation matrix.
    # Heart Disease Type Selector (Dropdown): Allows users to select a specific type of heart disease.

In [16]:
# Creates Correlation Heatmap
@pn.depends(ethnicity_selector_dropdown, age_range_selector_dropdown, states_selector_checkbox, heart_disease_type_selector_dropdown)
def update_plot(selected_ethnicity, selected_age_range, selected_states, selected_heart_disease_type):
    if not selected_states:
        return "Please select at least one state."
    
    # Filter data based on selected values
    filtered_data = filtered_df[
        (filtered_df['Ethnicity'] == selected_ethnicity) &
        (filtered_df['Age range'] == selected_age_range) &
        (filtered_df['Heart Disease Type'] == selected_heart_disease_type) &
        (filtered_df['US States'].isin(selected_states))
    ]
    
    if filtered_data.empty:
        return "No data available for the selected criteria."
    
    # Pivot the data to create a correlation matrix
    pivot_table = filtered_data.pivot_table(
        values='Data_Value/100_000 People', 
        index='Year', 
        columns='US States'
    )
    
    # Calculate the correlation matrix
    correlation_matrix = pivot_table.corr()

    # Create a heatmap
    heatmap = correlation_matrix.hvplot.heatmap(
        title=f'Correlation Matrix for {selected_heart_disease_type} Mortality Rates in {selected_ethnicity} ({selected_age_range})',
        cmap='viridis',
        width=800,
        height=600,
        rot=45,
        xrotation=45
    ).opts(colorbar=True)

    return heatmap

# Create a panel layout with widgets and the plot
dashboard = pn.Column(
    "## Cardiovascular Disease Mortality Rates State Correlation Heat Map Generator",
    pn.Row(ethnicity_selector_dropdown, pn.Column(age_range_selector_dropdown, heart_disease_type_selector_dropdown)),
    pn.Row(states_selector_checkbox),
    update_plot
)

# Serve the dashboard
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] Select(options=['American Indian/Alaska N...], value='American Indian/Alaska N...)
        [1] Column
            [0] Select(options=['Ages 35-64 years', ...], value='Ages 35-64 years')
            [1] Select(options=['All heart disease', ...], value='All heart disease')
    [2] Row
        [0] CheckBoxGroup(inline=True, name='Select States', options=['AK', 'AL', 'AR', ...])
    [3] ParamFunction(function, _pane=Markdown, defer_load=False)